### 1.1 Load Data Efficiently

In [1]:
import pandas as pd
import numpy as np
import polars as pl

In [ ]:

# Load CSV efficiently
df = pl.read_csv("transactions_dataset.csv", separator=";", try_parse_dates=True)


In [3]:
df.head()

date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
date,date,i64,i64,f64,i64,str,i64
2017-09-25,2017-09-25,2376333,188502,155.4432,3,"""online""",2732
2017-09-25,2017-09-25,2520527,835089,16.3944,3,"""at the store""",10279
2017-09-25,2017-09-25,2536756,1977896,365.76624,21,"""online""",6184
2017-09-25,2017-09-25,3028673,598816,8.59878,201,"""at the store""",4200
2017-09-25,2017-09-25,203377,2086861,1093.374,3,"""by phone""",7051


In [4]:
relation_df = pd.read_csv("sales_client.csv")
relation_df.head()

,client_id,quali_relation
0,188502,Difficult client
1,598816,Difficult client
2,564878,Difficult client
3,1883627,Difficult client
4,224223,Difficult client


In [5]:
# Convert relation_df to Polars DataFrame
relation_df_pl = pl.from_pandas(relation_df)

# Merge the datasets based on client_id
merged_df = df.join(relation_df_pl, on="client_id", how="left")

# Display the first few rows of the merged dataframe
print(merged_df.head())

C:\Users\DELL\AppData\Local\Temp\ipykernel_8524\3463571570.py:5: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  merged_df = df.join(relation_df_pl, on="client_id", how="left")


shape: (5, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ date_orde ┆ date_invo ┆ product_i ┆ client_id ┆ … ┆ quantity ┆ order_cha ┆ branch_id ┆ quali_rel │
│ r         ┆ ice       ┆ d         ┆ ---       ┆   ┆ ---      ┆ nnel      ┆ ---       ┆ ation     │
│ ---       ┆ ---       ┆ ---       ┆ i64       ┆   ┆ i64      ┆ ---       ┆ i64       ┆ ---       │
│ date      ┆ date      ┆ i64       ┆           ┆   ┆          ┆ str       ┆           ┆ str       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 2017-09-2 ┆ 2017-09-2 ┆ 2376333   ┆ 188502    ┆ … ┆ 3        ┆ online    ┆ 2732      ┆ Difficult │
│ 5         ┆ 5         ┆           ┆           ┆   ┆          ┆           ┆           ┆ client    │
│ 2017-09-2 ┆ 2017-09-2 ┆ 2520527   ┆ 835089    ┆ … ┆ 3        ┆ at the    ┆ 10279     ┆ Demanding │
│ 5         ┆ 5         ┆           ┆           ┆   ┆          ┆ store     ┆ 

### Profitability Index computation

In [76]:
# Add a year column to the dataframe
df = df.with_columns(
    pl.col("date_order").dt.year().alias("year")
)

# Group by client_id, order_channel, and year, and count unique dates and sum sales_net
sales_by_customer_channel_year = df.groupby(["client_id", "order_channel", "year"]).agg(
    [
        pl.col("date_order").n_unique().alias("sales_count"),
        pl.col("sales_net").sum().alias("sales_net_sum")
    ]
)

# Display the result
print(sales_by_customer_channel_year)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8524\3715135940.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  sales_by_customer_channel_year = df.groupby(["client_id", "order_channel", "year"]).agg(


shape: (646_580, 5)
┌───────────┬───────────────┬──────┬─────────────┬───────────────┐
│ client_id ┆ order_channel ┆ year ┆ sales_count ┆ sales_net_sum │
│ ---       ┆ ---           ┆ ---  ┆ ---         ┆ ---           │
│ i64       ┆ str           ┆ i32  ┆ u32         ┆ f64           │
╞═══════════╪═══════════════╪══════╪═════════════╪═══════════════╡
│ 2200107   ┆ at the store  ┆ 2017 ┆ 8           ┆ 2906.725832   │
│ 1891002   ┆ by phone      ┆ 2018 ┆ 12          ┆ 9939.878352   │
│ 993645    ┆ at the store  ┆ 2017 ┆ 1           ┆ 52.7666       │
│ 1146681   ┆ by phone      ┆ 2017 ┆ 1           ┆ 4058.32056    │
│ 417224    ┆ by phone      ┆ 2018 ┆ 1           ┆ 241.73        │
│ …         ┆ …             ┆ …    ┆ …           ┆ …             │
│ 902640    ┆ by phone      ┆ 2019 ┆ 17          ┆ 4323.61222    │
│ 1772380   ┆ by phone      ┆ 2019 ┆ 9           ┆ 3426.815233   │
│ 1971577   ┆ online        ┆ 2019 ┆ 7           ┆ 11194.092832  │
│ 431452    ┆ by phone      ┆ 2017 ┆ 8    

In [77]:
# Sort the dataframe by client_id and year
sales_by_customer_channel_year = sales_by_customer_channel_year.sort(["client_id", "year"])

# Display the sorted dataframe
print(sales_by_customer_channel_year)

shape: (646_580, 5)
┌───────────┬───────────────┬──────┬─────────────┬───────────────┐
│ client_id ┆ order_channel ┆ year ┆ sales_count ┆ sales_net_sum │
│ ---       ┆ ---           ┆ ---  ┆ ---         ┆ ---           │
│ i64       ┆ str           ┆ i32  ┆ u32         ┆ f64           │
╞═══════════╪═══════════════╪══════╪═════════════╪═══════════════╡
│ 6         ┆ by phone      ┆ 2017 ┆ 1           ┆ 1358.089878   │
│ 6         ┆ at the store  ┆ 2017 ┆ 1           ┆ 204.1296      │
│ 6         ┆ at the store  ┆ 2018 ┆ 2           ┆ 464.2228      │
│ 6         ┆ by phone      ┆ 2018 ┆ 3           ┆ 1133.901825   │
│ 6         ┆ by phone      ┆ 2019 ┆ 3           ┆ 2071.646974   │
│ …         ┆ …             ┆ …    ┆ …           ┆ …             │
│ 2274517   ┆ by phone      ┆ 2017 ┆ 4           ┆ 4383.780923   │
│ 2274517   ┆ by phone      ┆ 2018 ┆ 5           ┆ 5833.756603   │
│ 2274517   ┆ at the store  ┆ 2018 ┆ 5           ┆ 2032.074472   │
│ 2274517   ┆ at the store  ┆ 2019 ┆ 5    

In [78]:
# Add fixed hourly sales rate
sales_by_customer_channel_year = sales_by_customer_channel_year.with_columns(
    (pl.lit(80)).alias("fixed_hourly_sales_rate")
)

# Display the result
print(sales_by_customer_channel_year)

shape: (646_580, 6)
┌───────────┬───────────────┬──────┬─────────────┬───────────────┬─────────────────────────┐
│ client_id ┆ order_channel ┆ year ┆ sales_count ┆ sales_net_sum ┆ fixed_hourly_sales_rate │
│ ---       ┆ ---           ┆ ---  ┆ ---         ┆ ---           ┆ ---                     │
│ i64       ┆ str           ┆ i32  ┆ u32         ┆ f64           ┆ i32                     │
╞═══════════╪═══════════════╪══════╪═════════════╪═══════════════╪═════════════════════════╡
│ 6         ┆ by phone      ┆ 2017 ┆ 1           ┆ 1358.089878   ┆ 80                      │
│ 6         ┆ at the store  ┆ 2017 ┆ 1           ┆ 204.1296      ┆ 80                      │
│ 6         ┆ at the store  ┆ 2018 ┆ 2           ┆ 464.2228      ┆ 80                      │
│ 6         ┆ by phone      ┆ 2018 ┆ 3           ┆ 1133.901825   ┆ 80                      │
│ 6         ┆ by phone      ┆ 2019 ┆ 3           ┆ 2071.646974   ┆ 80                      │
│ …         ┆ …             ┆ …    ┆ …           ┆

In [79]:
# Join sales_by_customer_channel_year with relation_df_pl to add the quality of customer status
sales_by_customer_channel_year = sales_by_customer_channel_year.join(relation_df_pl, on="client_id", how="left")

# Display the result
print(sales_by_customer_channel_year.head())

shape: (5, 7)
┌───────────┬───────────────┬──────┬─────────────┬───────────────┬────────────────┬────────────────┐
│ client_id ┆ order_channel ┆ year ┆ sales_count ┆ sales_net_sum ┆ fixed_hourly_s ┆ quali_relation │
│ ---       ┆ ---           ┆ ---  ┆ ---         ┆ ---           ┆ ales_rate      ┆ ---            │
│ i64       ┆ str           ┆ i32  ┆ u32         ┆ f64           ┆ ---            ┆ str            │
│           ┆               ┆      ┆             ┆               ┆ i32            ┆                │
╞═══════════╪═══════════════╪══════╪═════════════╪═══════════════╪════════════════╪════════════════╡
│ 6         ┆ by phone      ┆ 2017 ┆ 1           ┆ 1358.089878   ┆ 80             ┆ Agreeable      │
│           ┆               ┆      ┆             ┆               ┆                ┆ client         │
│ 6         ┆ at the store  ┆ 2017 ┆ 1           ┆ 204.1296      ┆ 80             ┆ Agreeable      │
│           ┆               ┆      ┆             ┆               ┆           

C:\Users\DELL\AppData\Local\Temp\ipykernel_8524\2644061185.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  sales_by_customer_channel_year = sales_by_customer_channel_year.join(relation_df_pl, on="client_id", how="left")


In [80]:
# Sort the dataframe by client_id, order_channel, and year
sorted_sales_by_customer_channel_year = sales_by_customer_channel_year.sort(["client_id", "order_channel", "year"])

# Display the sorted dataframe
print(sorted_sales_by_customer_channel_year)

shape: (646_580, 7)
┌───────────┬───────────────┬──────┬─────────────┬───────────────┬────────────────┬────────────────┐
│ client_id ┆ order_channel ┆ year ┆ sales_count ┆ sales_net_sum ┆ fixed_hourly_s ┆ quali_relation │
│ ---       ┆ ---           ┆ ---  ┆ ---         ┆ ---           ┆ ales_rate      ┆ ---            │
│ i64       ┆ str           ┆ i32  ┆ u32         ┆ f64           ┆ ---            ┆ str            │
│           ┆               ┆      ┆             ┆               ┆ i32            ┆                │
╞═══════════╪═══════════════╪══════╪═════════════╪═══════════════╪════════════════╪════════════════╡
│ 6         ┆ at the store  ┆ 2017 ┆ 1           ┆ 204.1296      ┆ 80             ┆ Agreeable      │
│           ┆               ┆      ┆             ┆               ┆                ┆ client         │
│ 6         ┆ at the store  ┆ 2018 ┆ 2           ┆ 464.2228      ┆ 80             ┆ Agreeable      │
│           ┆               ┆      ┆             ┆               ┆     

In [81]:
# Categorize clients
sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.with_columns(
  pl.when(pl.col("quali_relation") == "Agreeable client")
    .then(pl.lit("Easy"))
    .otherwise(pl.lit("Difficult"))
    .alias("client_category")
)

# Display the result
print(sorted_sales_by_customer_channel_year)

shape: (646_580, 8)
┌───────────┬─────────────┬──────┬─────────────┬────────────┬────────────┬────────────┬────────────┐
│ client_id ┆ order_chann ┆ year ┆ sales_count ┆ sales_net_ ┆ fixed_hour ┆ quali_rela ┆ client_cat │
│ ---       ┆ el          ┆ ---  ┆ ---         ┆ sum        ┆ ly_sales_r ┆ tion       ┆ egory      │
│ i64       ┆ ---         ┆ i32  ┆ u32         ┆ ---        ┆ ate        ┆ ---        ┆ ---        │
│           ┆ str         ┆      ┆             ┆ f64        ┆ ---        ┆ str        ┆ str        │
│           ┆             ┆      ┆             ┆            ┆ i32        ┆            ┆            │
╞═══════════╪═════════════╪══════╪═════════════╪════════════╪════════════╪════════════╪════════════╡
│ 6         ┆ at the      ┆ 2017 ┆ 1           ┆ 204.1296   ┆ 80         ┆ Agreeable  ┆ Easy       │
│           ┆ store       ┆      ┆             ┆            ┆            ┆ client     ┆            │
│ 6         ┆ at the      ┆ 2018 ┆ 2           ┆ 464.2228   ┆ 80       

In [82]:
# Add sale_time_spent column based on the specified conditions
sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.with_columns(
  pl.when(pl.col("order_channel") == "by phone")
    .then(pl.when(pl.col("client_category") == "Easy").then(1).otherwise(2))
    .when(pl.col("order_channel") == "at the store")
    .then(4)
    .when(pl.col("order_channel").str.starts_with("during"))
    .then(4)
    .when(pl.col("order_channel") == "online")
    .then(0.5)
    .when(pl.col("order_channel") == "other")
    .then(0.5)
    .otherwise(pl.lit(None))
    .alias("sale_time_spent")
)

# Display the result
print(sorted_sales_by_customer_channel_year)

shape: (646_580, 9)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ fixed_hour ┆ quali_rela ┆ client_ca ┆ sale_time │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ ly_sales_r ┆ tion       ┆ tegory    ┆ _spent    │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ate        ┆ ---        ┆ ---       ┆ ---       │
│           ┆ str        ┆      ┆ u32        ┆   ┆ ---        ┆ str        ┆ str       ┆ f64       │
│           ┆            ┆      ┆            ┆   ┆ i32        ┆            ┆           ┆           │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6         ┆ at the     ┆ 2017 ┆ 1          ┆ … ┆ 80         ┆ Agreeable  ┆ Easy      ┆ 4.0       │
│           ┆ store      ┆      ┆            ┆   ┆            ┆ client     ┆           ┆           │
│ 6         ┆ at the     ┆ 2018 ┆ 2          ┆ … ┆ 80         ┆ Agreeab

In [83]:
# Create a new column in the sorted_sales_by_customer_channel_year dataframe to store the total sales calculated
sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.with_columns(
    pl.col("sales_net_sum").sum().over(["client_id", "year"]).alias("total_net_annual_sales")
)

# Display the result
print(sorted_sales_by_customer_channel_year)

shape: (646_580, 10)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ quali_rela ┆ client_cat ┆ sale_time ┆ total_net │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ tion       ┆ egory      ┆ _spent    ┆ _annual_s │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ales      │
│           ┆ str        ┆      ┆ u32        ┆   ┆ str        ┆ str        ┆ f64       ┆ ---       │
│           ┆            ┆      ┆            ┆   ┆            ┆            ┆           ┆ f64       │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6         ┆ at the     ┆ 2017 ┆ 1          ┆ … ┆ Agreeable  ┆ Easy       ┆ 4.0       ┆ 1562.2194 │
│           ┆ store      ┆      ┆            ┆   ┆ client     ┆            ┆           ┆ 78        │
│ 6         ┆ at the     ┆ 2018 ┆ 2          ┆ … ┆ Agreeable  ┆ Easy  

In [90]:
# Filter the dataframe for client_id 6
client_6_data = sorted_sales_by_customer_channel_year.filter(pl.col("client_id") == 2271274)

# Display the result
print(client_6_data)

shape: (5, 10)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ quali_rela ┆ client_cat ┆ sale_time ┆ total_net │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ tion       ┆ egory      ┆ _spent    ┆ _annual_s │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ales      │
│           ┆ str        ┆      ┆ u32        ┆   ┆ str        ┆ str        ┆ f64       ┆ ---       │
│           ┆            ┆      ┆            ┆   ┆            ┆            ┆           ┆ f64       │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2271274   ┆ at the     ┆ 2017 ┆ 4          ┆ … ┆ Agreeable  ┆ Easy       ┆ 4.0       ┆ 1024.7840 │
│           ┆ store      ┆      ┆            ┆   ┆ client     ┆            ┆           ┆ 75        │
│ 2271274   ┆ at the     ┆ 2018 ┆ 18         ┆ … ┆ Agreeable  ┆ Easy       ┆

In [91]:
# Calculate the gamma coefficient for each customer by year
gamma_coefficient_df = sorted_sales_by_customer_channel_year.groupby(["client_id", "year"]).agg(
    [
        ((pl.col("sale_time_spent") * pl.col("fixed_hourly_sales_rate") * pl.col("sales_count")).sum() / pl.col("total_net_annual_sales").first()).alias("gamma_coefficient")
    ]
)

# Display the result
print(gamma_coefficient_df)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8524\1894913612.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  gamma_coefficient_df = sorted_sales_by_customer_channel_year.groupby(["client_id", "year"]).agg(


shape: (379_284, 3)
┌───────────┬──────┬───────────────────┐
│ client_id ┆ year ┆ gamma_coefficient │
│ ---       ┆ ---  ┆ ---               │
│ i64       ┆ i32  ┆ f64               │
╞═══════════╪══════╪═══════════════════╡
│ 105404    ┆ 2018 ┆ 0.06074           │
│ 364911    ┆ 2018 ┆ 1.442986          │
│ 16317     ┆ 2019 ┆ 2.910537          │
│ 221600    ┆ 2017 ┆ 0.905554          │
│ 33682     ┆ 2018 ┆ 3.876363          │
│ …         ┆ …    ┆ …                 │
│ 91039     ┆ 2018 ┆ 1.0291            │
│ 1284933   ┆ 2019 ┆ 0.482618          │
│ 20656     ┆ 2019 ┆ 1.126314          │
│ 1237677   ┆ 2019 ┆ 0.450578          │
│ 1530751   ┆ 2018 ┆ 1.598981          │
└───────────┴──────┴───────────────────┘


In [92]:
# Join the gamma_coefficient_df with sorted_sales_by_customer_channel_year to add the gamma coefficient
sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.join(gamma_coefficient_df, on=["client_id", "year"], how="left")

# Display the result
print(sorted_sales_by_customer_channel_year)


shape: (646_580, 11)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ client_cat ┆ sale_time_ ┆ total_net ┆ gamma_coe │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ egory      ┆ spent      ┆ _annual_s ┆ fficient  │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ales      ┆ ---       │
│           ┆ str        ┆      ┆ u32        ┆   ┆ str        ┆ f64        ┆ ---       ┆ f64       │
│           ┆            ┆      ┆            ┆   ┆            ┆            ┆ f64       ┆           │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6         ┆ at the     ┆ 2017 ┆ 1          ┆ … ┆ Easy       ┆ 4.0        ┆ 1562.2194 ┆ 0.256046  │
│           ┆ store      ┆      ┆            ┆   ┆            ┆            ┆ 78        ┆           │
│ 6         ┆ at the     ┆ 2018 ┆ 2          ┆ … ┆ Easy       ┆ 4.0   

C:\Users\DELL\AppData\Local\Temp\ipykernel_8524\91879695.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.join(gamma_coefficient_df, on=["client_id", "year"], how="left")


In [93]:
# Calculate the total sales for each client
print(sorted_sales_by_customer_channel_year)


shape: (646_580, 11)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ client_cat ┆ sale_time_ ┆ total_net ┆ gamma_coe │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ egory      ┆ spent      ┆ _annual_s ┆ fficient  │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ales      ┆ ---       │
│           ┆ str        ┆      ┆ u32        ┆   ┆ str        ┆ f64        ┆ ---       ┆ f64       │
│           ┆            ┆      ┆            ┆   ┆            ┆            ┆ f64       ┆           │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6         ┆ at the     ┆ 2017 ┆ 1          ┆ … ┆ Easy       ┆ 4.0        ┆ 1562.2194 ┆ 0.256046  │
│           ┆ store      ┆      ┆            ┆   ┆            ┆            ┆ 78        ┆           │
│ 6         ┆ at the     ┆ 2018 ┆ 2          ┆ … ┆ Easy       ┆ 4.0   

In [95]:
# Add profitability index column
sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.with_columns(
    (100 - (pl.col("gamma_coefficient") * 100)).alias("profitability_index")
)

# Display the result
print(sorted_sales_by_customer_channel_year)

shape: (646_580, 12)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ sale_time_ ┆ total_net_ ┆ gamma_coe ┆ profitabi │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ spent      ┆ annual_sal ┆ fficient  ┆ lity_inde │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ es         ┆ ---       ┆ x         │
│           ┆ str        ┆      ┆ u32        ┆   ┆ f64        ┆ ---        ┆ f64       ┆ ---       │
│           ┆            ┆      ┆            ┆   ┆            ┆ f64        ┆           ┆ f64       │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6         ┆ at the     ┆ 2017 ┆ 1          ┆ … ┆ 4.0        ┆ 1562.21947 ┆ 0.256046  ┆ 74.395403 │
│           ┆ store      ┆      ┆            ┆   ┆            ┆ 8          ┆           ┆           │
│ 6         ┆ at the     ┆ 2018 ┆ 2          ┆ … ┆ 4.0        ┆ 1598.1

In [97]:
# Sort the dataframe by client_id, year, and profitability_index in descending order
sorted_sales_by_customer_channel_year = sorted_sales_by_customer_channel_year.sort(
    ["client_id", "year", "profitability_index"]
)

# Display the sorted dataframe
print(sorted_sales_by_customer_channel_year)

shape: (646_580, 12)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ sale_time_ ┆ total_net_ ┆ gamma_coe ┆ profitabi │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ spent      ┆ annual_sal ┆ fficient  ┆ lity_inde │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ es         ┆ ---       ┆ x         │
│           ┆ str        ┆      ┆ u32        ┆   ┆ f64        ┆ ---        ┆ f64       ┆ ---       │
│           ┆            ┆      ┆            ┆   ┆            ┆ f64        ┆           ┆ f64       │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6         ┆ by phone   ┆ 2017 ┆ 1          ┆ … ┆ 1.0        ┆ 1562.21947 ┆ 0.256046  ┆ 74.395403 │
│           ┆            ┆      ┆            ┆   ┆            ┆ 8          ┆           ┆           │
│ 6         ┆ at the     ┆ 2017 ┆ 1          ┆ … ┆ 4.0        ┆ 1562.2

In [100]:
print(sorted_sales_by_customer_channel_year.filter((pl.col("profitability_index") > 100) | (pl.col("profitability_index") < 0)))

shape: (161_022, 12)
┌───────────┬────────────┬──────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ client_id ┆ order_chan ┆ year ┆ sales_coun ┆ … ┆ sale_time_ ┆ total_net_ ┆ gamma_coe ┆ profitabi │
│ ---       ┆ nel        ┆ ---  ┆ t          ┆   ┆ spent      ┆ annual_sal ┆ fficient  ┆ lity_inde │
│ i64       ┆ ---        ┆ i32  ┆ ---        ┆   ┆ ---        ┆ es         ┆ ---       ┆ x         │
│           ┆ str        ┆      ┆ u32        ┆   ┆ f64        ┆ ---        ┆ f64       ┆ ---       │
│           ┆            ┆      ┆            ┆   ┆            ┆ f64        ┆           ┆ f64       │
╞═══════════╪════════════╪══════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 37        ┆ at the     ┆ 2019 ┆ 5          ┆ … ┆ 4.0        ┆ 950.259862 ┆ 1.68375   ┆ -68.37499 │
│           ┆ store      ┆      ┆            ┆   ┆            ┆            ┆           ┆ 6         │
│ 43        ┆ by phone   ┆ 2019 ┆ 1          ┆ … ┆ 2.0        ┆ 99.553

In [101]:
# Save the sorted_sales_by_customer_channel_year dataframe to a CSV file
sorted_sales_by_customer_channel_year.write_csv("sorted_sales_by_customer_channel_year.csv")

## 🔍 Step 2: Exploratory Data Analysis (EDA)

In [4]:
# Check memory usage and first rows
print(df.shape)
print(df.head())

(63319315, 8)
shape: (5, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬──────────┬────────────┬───────────┐
│ date_order ┆ date_invoi ┆ product_id ┆ client_id ┆ sales_net ┆ quantity ┆ order_chan ┆ branch_id │
│ ---        ┆ ce         ┆ ---        ┆ ---       ┆ ---       ┆ ---      ┆ nel        ┆ ---       │
│ date       ┆ ---        ┆ i64        ┆ i64       ┆ f64       ┆ i64      ┆ ---        ┆ i64       │
│            ┆ date       ┆            ┆           ┆           ┆          ┆ str        ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪══════════╪════════════╪═══════════╡
│ 2017-09-25 ┆ 2017-09-25 ┆ 2376333    ┆ 188502    ┆ 155.4432  ┆ 3        ┆ online     ┆ 2732      │
│ 2017-09-25 ┆ 2017-09-25 ┆ 2520527    ┆ 835089    ┆ 16.3944   ┆ 3        ┆ at the     ┆ 10279     │
│            ┆            ┆            ┆           ┆           ┆          ┆ store      ┆           │
│ 2017-09-25 ┆ 2017-09-25 ┆ 2536756    ┆ 1977896   ┆ 365.76624 

In [5]:
# Summary statistics
print(df.describe())


shape: (9, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ date_orde ┆ date_invo ┆ product_i ┆ … ┆ sales_net ┆ quantity  ┆ order_cha ┆ branch_i │
│ ---       ┆ r         ┆ ice       ┆ d         ┆   ┆ ---       ┆ ---       ┆ nnel      ┆ d        │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ f64       ┆ f64       ┆ ---       ┆ ---      │
│           ┆ str       ┆ str       ┆ f64       ┆   ┆           ┆           ┆ str       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 63319315  ┆ 63319314  ┆ 6.3319315 ┆ … ┆ 6.3319315 ┆ 6.3319315 ┆ 63319315  ┆ 6.331931 │
│           ┆           ┆           ┆ e7        ┆   ┆ e7        ┆ e7        ┆           ┆ 5e7      │
│ null_coun ┆ 0         ┆ 1         ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0         ┆ 0.0      │
│ t         ┆           ┆           ┆           ┆   ┆           ┆           ┆

In [6]:

# Check missing values
missing_values = df.null_count()
print(missing_values)

shape: (1, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬──────────┬────────────┬───────────┐
│ date_order ┆ date_invoi ┆ product_id ┆ client_id ┆ sales_net ┆ quantity ┆ order_chan ┆ branch_id │
│ ---        ┆ ce         ┆ ---        ┆ ---       ┆ ---       ┆ ---      ┆ nel        ┆ ---       │
│ u32        ┆ ---        ┆ u32        ┆ u32       ┆ u32       ┆ u32      ┆ ---        ┆ u32       │
│            ┆ u32        ┆            ┆           ┆           ┆          ┆ u32        ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪══════════╪════════════╪═══════════╡
│ 0          ┆ 1          ┆ 0          ┆ 0         ┆ 0         ┆ 0        ┆ 0          ┆ 0         │
└────────────┴────────────┴────────────┴───────────┴───────────┴──────────┴────────────┴───────────┘


In [7]:
# Fill missing invoice dates with estimated order date + 5 days (we could later on refine this estimate)
df = df.with_columns(
    pl.when(pl.col("date_invoice").is_null())
      .then(pl.col("date_order") + pl.duration(days=5))
      .otherwise(pl.col("date_invoice"))
      .alias("date_invoice")
)

In [8]:

# Check missing values
missing_values = df.null_count()
print(missing_values)

shape: (1, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬──────────┬────────────┬───────────┐
│ date_order ┆ date_invoi ┆ product_id ┆ client_id ┆ sales_net ┆ quantity ┆ order_chan ┆ branch_id │
│ ---        ┆ ce         ┆ ---        ┆ ---       ┆ ---       ┆ ---      ┆ nel        ┆ ---       │
│ u32        ┆ ---        ┆ u32        ┆ u32       ┆ u32       ┆ u32      ┆ ---        ┆ u32       │
│            ┆ u32        ┆            ┆           ┆           ┆          ┆ u32        ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪══════════╪════════════╪═══════════╡
│ 0          ┆ 0          ┆ 0          ┆ 0         ┆ 0         ┆ 0        ┆ 0          ┆ 0         │
└────────────┴────────────┴────────────┴───────────┴───────────┴──────────┴────────────┴───────────┘


In [9]:

# Distribution of order channels
print(df["order_channel"].value_counts())


shape: (5, 2)
┌─────────────────────────────────┬──────────┐
│ order_channel                   ┆ count    │
│ ---                             ┆ ---      │
│ str                             ┆ u32      │
╞═════════════════════════════════╪══════════╡
│ during the visit of a sales re… ┆ 29313    │
│ other                           ┆ 58735    │
│ at the store                    ┆ 32114196 │
│ online                          ┆ 5651866  │
│ by phone                        ┆ 25465205 │
└─────────────────────────────────┴──────────┘


In [ ]:
# Convert to pandas for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Example: convert to pandas for visualization
df_pd = df.to_pandas()

# Plot the distribution of a column
sns.histplot(df_pd['sales_net'])
plt.show()

# Correlation heatmap
sns.heatmap(df_pd.corr(), annot=True, cmap='coolwarm')
plt.show()


KeyboardInterrupt: 

In [10]:
# Check sales trends over time
df.groupby("date_order").agg(pl.col("sales_net").sum()).sort("date_order").plot()

C:\Users\DELL\AppData\Local\Temp\ipykernel_23528\4207683701.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df.groupby("date_order").agg(pl.col("sales_net").sum()).sort("date_order").plot()


:Curve   [index]   (sales_net)

## 📊 Step 3: Feature Engineering for Churn Prediction

In [11]:
# Get latest order date
latest_date = df["date_order"].max()

# Compute RFM features
rfm = df.group_by("client_id").agg([
    (latest_date - pl.col("date_order").max()).dt.days().alias("recency"),
    pl.len().alias("frequency"),
    pl.col("sales_net").sum().alias("monetary_value"),
    pl.col("sales_net").mean().alias("avg_order_value"),
    pl.col("quantity").mean().alias("avg_quantity"),
])


C:\Users\DELL\AppData\Local\Temp\ipykernel_23528\2982527216.py:6: DeprecationWarning: `days` is deprecated. It has been renamed to `total_days`.
  (latest_date - pl.col("date_order").max()).dt.days().alias("recency"),


In [12]:
rfm.head()

client_id,recency,frequency,monetary_value,avg_order_value,avg_quantity
i64,i64,u32,f64,f64,f64
1071508,10,187,33309.145404,178.123772,30.540107
193110,10,342,38834.576806,113.551394,504.280702
1327543,2,1165,106312.721628,91.255555,30.553648
93073,82,33,14082.555892,426.744118,93.0
1146771,2,114,43667.468503,383.047969,47.631579


In [13]:
# Compute days between orders per client
df = df.sort(["client_id", "date_order"])

df = df.with_columns(
    pl.col("date_order").diff().over("client_id").dt.days().alias("days_between_orders")
)

# Aggregate order patterns per client
time_features = df.group_by("client_id").agg([
    pl.col("days_between_orders").mean().alias("avg_days_between_orders"),
    pl.col("days_between_orders").std().alias("std_days_between_orders")
])

# Merge RFM & time-based features
rfm = rfm.join(time_features, on="client_id", how="left")

rfm.head()


C:\Users\DELL\AppData\Local\Temp\ipykernel_23528\1373336219.py:5: DeprecationWarning: `days` is deprecated. It has been renamed to `total_days`.
  pl.col("date_order").diff().over("client_id").dt.days().alias("days_between_orders")
C:\Users\DELL\AppData\Local\Temp\ipykernel_23528\1373336219.py:15: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  rfm = rfm.join(time_features, on="client_id", how="left")


client_id,recency,frequency,monetary_value,avg_order_value,avg_quantity,avg_days_between_orders,std_days_between_orders
i64,i64,u32,f64,f64,f64,f64,f64
1071508,10,187,33309.145404,178.123772,30.540107,3.795699,8.994364
193110,10,342,38834.576806,113.551394,504.280702,2.093842,6.118198
1327543,2,1165,106312.721628,91.255555,30.553648,0.621134,1.959153
93073,82,33,14082.555892,426.744118,93.0,14.1875,37.538314
1146771,2,114,43667.468503,383.047969,47.631579,4.610619,13.250629


In [14]:
# Compute churn label (no orders in the last year)
rfm = rfm.with_columns(
    (pl.col("recency") > 365).cast(pl.Int8()).alias("churn")
)

# Check class distribution
print(rfm["churn"].value_counts())

shape: (2, 2)
┌───────┬────────┐
│ churn ┆ count  │
│ ---   ┆ ---    │
│ i8    ┆ u32    │
╞═══════╪════════╡
│ 0     ┆ 142228 │
│ 1     ┆ 28361  │
└───────┴────────┘


## 🧠 Step 4: Train a Churn Prediction Model


In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb




In [16]:
import numpy as np
from collections import Counter

# Check class distribution
class_counts = Counter(y)
print(f"Class distribution: {class_counts}")
print(f"Minority class: {min(class_counts, key=class_counts.get)}")

Class distribution: Counter({np.int8(0): 142228, np.int8(1): 28361})
Minority class: 1


In [20]:
rfm.null_count()

client_id,recency,frequency,monetary_value,avg_order_value,avg_quantity,avg_days_between_orders,std_days_between_orders,churn
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,6310,13334,0


In [21]:
rfm.count()

client_id,recency,frequency,monetary_value,avg_order_value,avg_quantity,avg_days_between_orders,std_days_between_orders,churn
u32,u32,u32,u32,u32,u32,u32,u32,u32
170589,170589,170589,170589,170589,170589,164279,157255,170589


In [ ]:
rfm = rfm.fill_null(0)


In [23]:
rfm.null_count()

client_id,recency,frequency,monetary_value,avg_order_value,avg_quantity,avg_days_between_orders,std_days_between_orders,churn
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [48]:
rfm.head()

client_id,recency,frequency,monetary_value,avg_order_value,avg_quantity,avg_days_between_orders,std_days_between_orders,churn
i64,i64,i64,f64,f64,f64,f64,f64,i8
1071508,10,187,33309.145404,178.123772,30.540107,3.795699,8.994364,0
193110,10,342,38834.576806,113.551394,504.280702,2.093842,6.118198,0
1327543,2,1165,106312.721628,91.255555,30.553648,0.621134,1.959153,0
93073,82,33,14082.555892,426.744118,93.0,14.1875,37.538314,0
1146771,2,114,43667.468503,383.047969,47.631579,4.610619,13.250629,0


In [25]:
# Convert Polars dataframe to NumPy for ML
X = rfm.drop(["client_id", "churn"]).to_numpy()
y = rfm["churn"].to_numpy()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to generate synthetic minority samples
smote = SMOTE(sampling_strategy=0.7, random_state=42)  # Upsample minority class to 50% of majority
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"New class distribution: {Counter(y_train_resampled)}")
print(f"New minority class: {min(Counter(y_train_resampled), key=Counter(y_train_resampled).get)}")

New class distribution: Counter({np.int8(0): 113853, np.int8(1): 79697})
New minority class: 1


In [28]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# LightGBM with class weights
lgbm = lgb.LGBMClassifier(
    n_estimators=500, 
    learning_rate=0.05, 
    max_depth=5, 
    class_weight="balanced"  # Auto-weighting based on class distribution
)

# Train model
lgbm.fit(X_train_resampled, y_train_resampled)

# Predictions
y_pred = lgbm.predict(X_test)


[LightGBM] [Info] Number of positive: 79697, number of negative: 113853
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 193550, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [29]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(classification_report(y_test, y_pred))


Model Accuracy: 0.9985
F1 Score: 0.9957
ROC AUC Score: 0.9978
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28375
           1       0.99      1.00      1.00      5743

    accuracy                           1.00     34118
   macro avg       1.00      1.00      1.00     34118
weighted avg       1.00      1.00      1.00     34118



#### ✅ 1️⃣ Hyperparameter Tuning with GridSearchCV
<>

In [30]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'num_leaves': [20, 31, 40]
}

# Initialize the model
lgbm = lgb.LGBMClassifier(class_weight="balanced")

# Grid search with 3-fold cross-validation
grid_search = GridSearchCV(
    estimator=lgbm, 
    param_grid=param_grid, 
    scoring='roc_auc', 
    cv=3, 
    n_jobs=-1, 
    verbose=2
)

# Fit on resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Best parameters
print(f"Best Parameters: {grid_search.best_params_}")

# Train best model
best_lgbm = grid_search.best_estimator_


Fitting 3 folds for each of 81 candidates, totalling 243 fits
[LightGBM] [Info] Number of positive: 79697, number of negative: 113853
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 193550, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [37]:
import pandas as pd

# Generate final predictions
y_pred_proba = best_lgbm.predict_proba(X_test)[:, 1]  # Probabilities for churn class

# Predictions
y_pred = best_lgbm.predict(X_test)


In [38]:
# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.9986
F1 Score: 0.9959
ROC AUC Score: 0.9979
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28375
           1       0.99      1.00      1.00      5743

    accuracy                           1.00     34118
   macro avg       1.00      1.00      1.00     34118
weighted avg       1.00      1.00      1.00     34118



In [39]:
import joblib

joblib.dump(best_lgbm, 'best_lgbm_model.pkl')  # Save model

['best_lgbm_model.pkl']

In [46]:
X_test

array([[2.40000000e+01, 4.00000000e+00, 0.00000000e+00, ...,
        3.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e+00, 2.66000000e+02, 6.80226525e+04, ...,
        2.17443609e+01, 2.71698113e+00, 6.68438184e+00],
       [5.45000000e+02, 6.00000000e+00, 2.79616980e+02, ...,
        4.30000000e+01, 5.60000000e+00, 7.95612971e+00],
       ...,
       [1.20000000e+01, 3.80000000e+01, 6.70040352e+03, ...,
        1.08947368e+01, 1.52702703e+01, 5.10281233e+01],
       [3.27000000e+02, 1.00000000e+00, 4.88581333e+01, ...,
        7.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 4.19000000e+02, 2.86055481e+04, ...,
        4.42076372e+01, 7.53588517e-01, 5.64446624e+00]])

In [40]:
df_results = pd.DataFrame({
    "client_id": X_test['client_id'],  # Assuming client_id is a column
    "churn_probability": y_pred_proba
})

# Save as CSV for Tableau
df_results.to_csv("churn_predictions.csv", index=False)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## ✅ Deploying the Model to Tableau (POC)
In this final step, we will deploy the churn prediction model to Tableau for interactive visualization. We will use the CSV file generated in the previous step to create a dashboard that allows users to explore the churn probability for each client.
    